In [6]:
from dotenv import load_dotenv
load_dotenv()

True

# Retrieval

In [11]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
caminhos = [
    'arquivos/Explorando o Universo das IAs com Hugging Face.pdf',
    'arquivos/Explorando a API da OpenAI.pdf',
    'arquivos/Explorando a API da OpenAI.pdf',
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=['\n\n', '\n', '.', ' ', '']
)

documents = recur_split.split_documents(paginas)
len(paginas)

265

### Modificando metadata

In [14]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/', '')
    doc.metadata['doc_id'] = i

In [15]:
print(documents[0].metadata)
print(documents[500].metadata)

{'source': 'Explorando o Universo das IAs com Hugging Face.pdf', 'page': 0, 'doc_id': 0}
{'source': 'Explorando a API da OpenAI.pdf', 'page': 79, 'doc_id': 500}


In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embeddings_model = OpenAIEmbeddings()

### Criando VectorStore

In [17]:
diretorio = 'arquivos/chroma_retrieval_bd'

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

## Semantic Search

In [8]:
pergunta = 'O que é a OpenAI?'

docs = vectordb.similarity_search(pergunta, k=3)

for doc in docs:
    print(doc.page_content)
    print(f'================={doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam
================={'doc_id': 283, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a ma

### Limitações do semantic search

#### Trechos com muito alta similaridade e pouca infomação adicional

In [ ]:
pergunta = 'O que é a OpenAI?'

docs = vectordb.similarity_search(pergunta, k=3)

for doc in docs:
    print(doc.page_content)
    print(f'================={doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam
==========={'doc_id': 283, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira 

In [9]:
pergunta = 'O que é a apostila de Hugging face fala sobre a OpenAI e o ChatGpt?'

docs = vectordb.similarity_search(pergunta, k=3)

for doc in docs:
    print(doc.page_content)
    print(f'================={doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1, disponível no link a seguir:
================={'doc_id': 86, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
01. O que é Hugging Face?
Bem-vindos ao curso de Hugging Face da Asimov Academy!
Neste curso, vamos explorar as principais utilidades da plataforma de IAHugging Face. Aprenderemos
como utilizar a plataforma ao máximo, e como incorporar as bibliotecas de Python do Hugging Face
aos nossos scripts de Python.
Afinal, o que é Hugging Face?
A Hugging Face é uma empr

## MMR

In [10]:
pergunta = 'O que é a OpenAI?'

docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)
for doc in docs:
    print(doc.page_content)
    print(f'================={doc.metadata}\n\n')

Explorando a API da OpenAI
01. Bem-vindos ao curso Explorando a API da OpenAI
Olá e bem-vindos ao nosso curso “Explorando a API da OpenAI”! Estou super animado por ter vocês
aqui e mal posso esperar para explorarmos juntos esse universo fascinante da programação e da
inteligência artificial.
Vocês já devem ter percebido que a IA está mudando o jogo em muitos campos, certo? É incrível como
ela está transformando a maneira como lidamos com informações e realiza tarefas que antes pareciam
================={'doc_id': 283, 'page': 4, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
Guias
Também está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da
OpenAI.
Asimov Academy 9
================={'doc_id': 299, 'page': 9, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo 

## Filtragem

In [12]:
pergunta = 'O que a apostila de HuggingFace fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(pergunta, k=3, filter={'source': 'Explorando o Universo das IAs com Hugging Face.pdf'})

for doc in docs:
    print(doc.page_content)
    print(f'================{doc.metadata}\n\n')


Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1, disponível no link a seguir:
================{'doc_id': 86, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots) 29
Acessando o modelo . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 29
Ajustando o template da mensagem . . . . . . . . . . . . . . . . . . . . . . . . . . . 30
Ajustando parâmetros do modelo . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 30
07. Criando uma estru

In [14]:
pergunta = 'O que a apostila de HuggingFace fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(
    pergunta, 
    k=3, 
    filter={'source': {'$in':['Explorando o Universo das IAs com Hugging Face.pdf', 'Explorando a API da OpenAI.pdf']}})

for doc in docs:
    print(doc.page_content)
    print(f'================{doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
06. Modelos de conversação e geração de texto (chatbots)
Vamos aprender agora a usar modelos de conversação, que funcionam de forma parecida com chatbots
como ChatGPT . Na realidade, estes modelos estão classificados na tarefa de geração de texto (text
generation) no Hugging Face:
Figure 16: Página principal da tarefatext-generation do Hugging Face.
Vamos utilizar o modelo Felladrin/Llama-68M-Chat-v1, disponível no link a seguir:
================{'doc_id': 86, 'page': 29, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela
empresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:
• GPT-4: Trata-se de um modelo de linguagem de grande escala (LLM) multimodal, que ace

In [17]:
pergunta = 'O que a apostila de HuggingFace fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(
    pergunta, 
    k=3, 
    filter={'$and':
        [
            {'source': {'$in':['Explorando o Universo das IAs com Hugging Face.pdf', 'Explorando a API da OpenAI.pdf']}},
            {'page': {'$in': [10, 11, 12, 13]}}
        ],
        })

for doc in docs:
    print(doc.page_content)
    print(f'================{doc.metadata}\n\n')

Explorando o Universo das IAs com Hugging Face
Nas próximas aulas, vamos aprender aos poucos a usar toda a informação contida nesta plataforma!
Asimov Academy 13
================{'doc_id': 42, 'page': 13, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
Figure 6: Página dos Spaces.
Cada Space funciona de acordo com o modelo de IA que está rodando nele. Por debaixo dos panos,
estes webapps utilizam a biblioteca gradio, que possui relação próxima com o Hugging Face.
Asimov Academy 11
================{'doc_id': 40, 'page': 11, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}


Explorando o Universo das IAs com Hugging Face
Figure 7: Exemplo de webapp de IA rodando em um Space.
Asimov Academy 12
================{'doc_id': 41, 'page': 12, 'source': 'Explorando o Universo das IAs com Hugging Face.pdf'}




## LLM Aided Retrieval

In [7]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

In [8]:
metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Deve ter o valor de: \
            Explorando o Universo das IAs com Hugging Face.pdf ou Explorando a API da OpenAI.pdf',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto se origina',
        type='integer'
    )
]

In [18]:
document_description = 'Apostila de curso'
llm = OpenAI()
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb, 
    document_description,
    metadata_info,
    verbose=True
)

In [23]:
pergunta = 'O que a apostila de HuggingFace fala sobre a OpenAI e o ChatGPT?'

docs = retriever.get_relevant_documents(pergunta)

for doc in docs:
    print(doc.page_content)
    print(f'================{doc.metadata}\n\n')

[chain/start] [retriever:SelfQueryRetriever > chain:query_constructor] Entering Chain run with input:
{
  "query": "O que a apostila de HuggingFace fala sobre a OpenAI e o ChatGPT?"
}
[chain/start] [retriever:SelfQueryRetriever > chain:query_constructor > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "O que a apostila de HuggingFace fala sobre a OpenAI e o ChatGPT?"
}
[chain/end] [retriever:SelfQueryRetriever > chain:query_constructor > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:SelfQueryRetriever > chain:query_constructor > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to compare to document contents\n    \"filter\": 

In [27]:
pergunta = 'Quais detalhes são descritos na página 44 da apostila Explorando a API da OpenAI.pdf?'

docs = retriever.get_relevant_documents(pergunta)

for doc in docs:
    print(doc.page_content)
    print(f'================{doc.metadata}\n\n')

Explorando a API da OpenAI
Usos comuns
Alguns casos de uso comuns nos quais o fine-tuning pode aprimorar os resultados incluem:
• Definir o estilo, tom, formato ou outros aspectos qualitativos.
• Aumentar a confiabilidade na geração de uma saída desejada.
• Corrigir falhas ao seguir instruções complexas.
• Lidar com muitos casos atípicos de maneiras específicas.
• Aprender uma nova habilidade ou realizar uma tarefa difícil de articular em uma instrução.
================{'doc_id': 402, 'page': 44, 'source': 'Explorando a API da OpenAI.pdf'}


Explorando a API da OpenAI
Usos comuns
Alguns casos de uso comuns nos quais o fine-tuning pode aprimorar os resultados incluem:
• Definir o estilo, tom, formato ou outros aspectos qualitativos.
• Aumentar a confiabilidade na geração de uma saída desejada.
• Corrigir falhas ao seguir instruções complexas.
• Lidar com muitos casos atípicos de maneiras específicas.
• Aprender uma nova habilidade ou realizar uma tarefa difícil de articular em uma instr

In [ ]:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'Quais detalhes são descritos na página 44 da apostila Explorando a API da OpenAI.pdf?'

docs = retriever.get_relevant_documents(pergunta)

for doc in docs:
    print(doc.page_content)
    print(f'================{doc.metadata}\n\n')

[chain/start] [retriever:SelfQueryRetriever > chain:query_constructor] Entering Chain run with input:
{
  "query": "Quais detalhes são descritos na página 44 da apostila Explorando a API da OpenAI.pdf?"
}
[chain/start] [retriever:SelfQueryRetriever > chain:query_constructor > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "Quais detalhes são descritos na página 44 da apostila Explorando a API da OpenAI.pdf?"
}
[chain/end] [retriever:SelfQueryRetriever > chain:query_constructor > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:SelfQueryRetriever > chain:query_constructor > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to comp

In [28]:
import json

_json = '{\n    \"query\": \"detalhes\",\n    \"filter\": \"and(eq(\\\"source\\\", \\\"Explorando a API da OpenAI.pdf\\\"), eq(\\\"page\\\", 44))\"\n}'
print(json.dumps(json.loads(_json), indent=4))

{
    "query": "detalhes",
    "filter": "and(eq(\"source\", \"Explorando a API da OpenAI.pdf\"), eq(\"page\", 44))"
}
